In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px

In [3]:
df=pd.read_csv("datasets/AB_NYC_2019.csv")

Data Inspection
1. Check the features that are listed in your dataset. Consider the different values, what
measurements they each represent and if there is any information missing about the
measurement of those values. This may require some research.
2. Print a portion of the dataset (or all if it’s a small dataset) to familiarize yourself with the
information in the dataset.
3. What is the shape of your data?

In [4]:
#2. Print a portion of the dataset (or all if it’s a small dataset) to familiarize yourself with the information in the dataset.
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [5]:
#3. Shape of data
df.shape

(48895, 16)

Check the features that are listed in your dataset. Consider the different values, what measurements they each represent and if there is any information missing about the measurement of those values. This may require some research.

In [6]:
df.drop(['id', 'host_id'],axis=1).describe()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [7]:
df.host_id.nunique()

37457

In [49]:
df.neighbourhood_group.value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

#### Missing values

In [8]:
df.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [9]:
df_missing=df.copy()
#dropping irrelevant columns
df_missing.drop(['host_id','host_name'], axis=1, inplace=True)
#replacing missing values
df_missing[(df_missing.last_review.isnull()) & (df_missing.reviews_per_month.isnull())] = df_missing[(df_missing.last_review.isnull()) & (df_missing.reviews_per_month.isnull())].replace(np.nan,0)
#dropping irrelevant columns
df_missing.drop(['last_review'], axis=1, inplace=True)
#dropping rows with NaN in listing names
df_missing.dropna(subset = ['name'], inplace= True)
df_missing['name'] = df_missing.name.astype('str')
#feature encoding
df_encode = df_missing.copy(deep=True)
df_encode = pd.get_dummies(df_encode, columns = ['neighbourhood_group'],
                                        prefix = 'NG',drop_first=True)
df_encode.drop(['neighbourhood'], axis=1, inplace=True)
df_encode['room_type']=pd.factorize(df_encode.room_type)[0]
#truncate minimum nights
from scipy.stats.mstats import winsorize
df_win = df_encode.copy(deep=True)
df_win['minimum_nights'] = winsorize(df_win['minimum_nights'], limits=(0, 0.05))
#to remove the skeweness of the price
df_log = df_win.copy(deep=True)
df_log['price'] =  np.log1p(df_log['price'])
df_all = df_log.copy(deep=True)

In [75]:
df_encode['room_type'].value_counts()

1    25404
0    22325
2     1160
Name: room_type, dtype: int64

In [10]:
df_all.head()

,id,name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,NG_Brooklyn,NG_Manhattan,NG_Queens,NG_Staten Island
0,2539,Clean & quiet apt home by the park,40.64749,-73.97237,0,5.010635,1,9,0.21,6,365,1,0,0,0
1,2595,Skylit Midtown Castle,40.75362,-73.98377,1,5.420535,1,45,0.38,2,355,0,1,0,0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,40.80902,-73.94190,0,5.017280,3,0,0.00,1,365,0,1,0,0
3,3831,Cozy Entire Floor of Brownstone,40.68514,-73.95976,1,4.499810,1,270,4.64,1,194,1,0,0,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,40.79851,-73.94399,1,4.394449,10,9,0.10,1,0,0,1,0,0


In [62]:
df_all=df_all.rename(columns={"NG_Staten Island":"NG_Staten_Island"})

In [63]:
df_all

,id,name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,NG_Brooklyn,NG_Manhattan,NG_Queens,NG_Staten_Island
0,2539,Clean & quiet apt home by the park,40.64749,-73.97237,0,5.010635,1,9,0.21,6,365,1,0,0,0
1,2595,Skylit Midtown Castle,40.75362,-73.98377,1,5.420535,1,45,0.38,2,355,0,1,0,0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,40.80902,-73.94190,0,5.017280,3,0,0.00,1,365,0,1,0,0
3,3831,Cozy Entire Floor of Brownstone,40.68514,-73.95976,1,4.499810,1,270,4.64,1,194,1,0,0,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,40.79851,-73.94399,1,4.394449,10,9,0.10,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,40.67853,-73.94995,0,4.262680,2,0,0.00,2,9,1,0,0,0
48891,36485057,Affordable room in Bushwick/East Williamsburg,40.70184,-73.93317,0,3.713572,4,0,0.00,2,36,1,0,0,0
48892,36485431,Sunny Studio at Historical Neighborhood,40.81475,-73.94867,1,4.753590,10,0,0.00,1,27,0,1,0,0
48893,36485609,43rd St. Time Square-cozy single bed,40.75751,-73.99112,2,4.025352,1,0,0.00,6,2,0,1,0,0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import metrics

In [52]:
#linear regression model
lr_model = LinearRegression()

In [64]:
y = df_all['price']
X = df_all.drop(['price','name','id','latitude','longitude'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [65]:
lr_model.fit(X_train, y_train)

LinearRegression()

In [66]:
lr_model_yhat = lr_model.predict(X_test)

In [67]:
#Create Model
model = LinearRegression()
model.fit(X_train, y_train)
predict = model.predict(X_test)

print('\nOriginal Model')
print('\nMethod: Linear Regression')   

#Coefficents
print('\nIntercept: {:.2f}'.format(float(model.intercept_)))
coeff_table=pd.DataFrame(np.transpose(model.coef_),df_all.drop(['price','id','latitude','longitude','name'],axis=1).columns,columns=['Coefficients'])
print(coeff_table)
    
#R2,MAE,MSE and RMSE
print('\nR2: {:.2f}'.format(metrics.r2_score(y_test,predict)))
print('Mean Absolute Error: {:.2f}'.format(metrics.mean_absolute_error(y_test, predict)))  
print('Mean Squared Error: {:.2f}'.format(metrics.mean_squared_error(y_test, predict)))  
print('Root Mean Squared Error: {:.2f}'.format(np.sqrt(metrics.mean_squared_error(y_test, predict)))) 


Original Model

Method: Linear Regression

Intercept: 3.96
                                Coefficients
room_type                           0.545178
minimum_nights                     -0.008256
number_of_reviews                  -0.000411
reviews_per_month                  -0.021046
calculated_host_listings_count      0.000988
availability_365                    0.000671
NG_Brooklyn                         0.336968
NG_Manhattan                        0.688488
NG_Queens                           0.169577
NG_Staten_Island                    0.042190

R2: 0.33
Mean Absolute Error: 0.40
Mean Squared Error: 0.32
Root Mean Squared Error: 0.57


In [68]:
#Forecast Table
predict2 = predict.T
diff = predict2-y_test
FcstTble=pd.DataFrame({'Actual':y_test,'Predicted':predict2.round(1),'Difference':diff.round(1)})
print('\nForecast Table')
FcstTble.head(10)


Forecast Table


,Actual,Predicted,Difference
39330,4.615121,4.8,0.2
13710,4.615121,4.7,0.1
31398,3.583519,4.2,0.6
20780,4.110874,4.8,0.7
16962,5.252273,5.2,-0.1
40330,4.330733,4.3,-0.1
5956,6.893656,4.8,-2.1
35899,4.836282,4.8,-0.0
44940,3.663562,4.4,0.7
6182,5.525453,4.8,-0.7


In [69]:
import pickle

In [70]:
pickle.dump(model,open('model.pkl','wb'))

In [71]:
df_all.columns

Index(['id', 'name', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'NG_Brooklyn',
       'NG_Manhattan', 'NG_Queens', 'NG_Staten_Island'],
      dtype='object')

In [72]:
df_all.room_type.value_counts()

1    25404
0    22325
2     1160
Name: room_type, dtype: int64

In [73]:
X.describe()

,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,NG_Brooklyn,NG_Manhattan,NG_Queens,NG_Staten_Island
count,48889.000000,48889.000000,48889.000000,48889.000000,48889.000000,48889.000000,48889.000000,48889.000000,48889.000000,48889.000000
mean,0.567081,5.953098,23.276443,1.091025,7.144736,112.780789,0.411176,0.442983,0.115895,0.007630
std,0.541258,8.674930,44.552835,1.597346,32.954471,131.617539,0.492052,0.496744,0.320102,0.087014
min,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,3.000000,5.000000,0.370000,1.000000,45.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,5.000000,24.000000,1.580000,2.000000,227.000000,1.000000,1.000000,0.000000,0.000000
max,2.000000,30.000000,629.000000,58.500000,327.000000,365.000000,1.000000,1.000000,1.000000,1.000000
